In [111]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", '3.0.0', ignore_verifications=True)

from datasets import get_dataset_split_names
get_dataset_split_names("cnn_dailymail", '3.0.0')

C:\Users\kayir\anaconda3\envs\top2vec\lib\site-packages\datasets\load.py:2516: FutureWarning: 'ignore_verifications' was deprecated in favor of 'verification_mode' in version 2.9.1 and will be removed in 3.0.0.
You can remove this warning by passing 'verification_mode=no_checks' instead.
  warnings.warn(


['train', 'validation', 'test']

In [112]:
import pandas as pd
import re

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('universal_tagset')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn. metrics import adjusted_rand_score
import string

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kayir\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\kayir\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\kayir\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [113]:
dataset = load_dataset("cnn_dailymail", '3.0.0', split="train")
dataframe = pd.DataFrame(dataset)
df = dataframe.loc[0:9999]

articles = df["article"].tolist()

In [114]:
clean_articles = []
for article in articles:
  text = re.sub(r"NEW:", "", article)
  text = text.lower() # нижний регистр
  text_list_nltk = word_tokenize(text) # токенизация
  stop_words = stopwords.words('english')
  text_clean = [word for word in text_list_nltk if word not in stop_words and word[0].isalpha()] # чистим от стоп-слов и пунктуации
  text_clean = ' '.join(text_clean)
  clean_articles.append(text_clean)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



TypeError: 'NoneType' object is not subscriptable

In [201]:
print(clean_articles[0:10])

["london england reuters harry potter star daniel radcliffe gains access reported million million fortune turns monday insists money wo n't cast spell daniel radcliffe harry potter harry potter order phoenix disappointment gossip columnists around world young actor says plans fritter cash away fast cars drink celebrity parties n't plan one people soon turn suddenly buy massive sports car collection something similar told australian interviewer earlier month n't think particularly extravagant things like buying things cost pounds books cds dvds radcliffe able gamble casino buy drink pub see horror film hostel part ii currently six places number one movie uk box office chart details mark landmark birthday wraps agent publicist comment plans definitely sort party said interview hopefully none reading radcliffe earnings first five potter films held trust fund able touch despite growing fame riches actor says keeping feet firmly ground people always looking say star goes rails told reporter

In [202]:
vectorizer = TfidfVectorizer(
                                max_features = 100,
                                max_df = 0.8,
                                min_df = 5,
                                ngram_range = (1,3),
                                stop_words = "english"
                            )

In [203]:
vectors = vectorizer.fit_transform(clean_articles)

In [119]:
feature_names = vectorizer.get_feature_names_out()

dense = vectors.todense()
denselist = dense.tolist()

all_keywords = []

for description in denselist:
    x = 0
    keywords = []
    for word in description:
        if word > 0:
            keywords.append(feature_names[x])
        x += 1
    all_keywords.append(keywords)


In [204]:
print(clean_articles[0])
print(all_keywords[0])

london england reuters harry potter star daniel radcliffe gains access reported million million fortune turns monday insists money wo n't cast spell daniel radcliffe harry potter harry potter order phoenix disappointment gossip columnists around world young actor says plans fritter cash away fast cars drink celebrity parties n't plan one people soon turn suddenly buy massive sports car collection something similar told australian interviewer earlier month n't think particularly extravagant things like buying things cost pounds books cds dvds radcliffe able gamble casino buy drink pub see horror film hostel part ii currently six places number one movie uk box office chart details mark landmark birthday wraps agent publicist comment plans definitely sort party said interview hopefully none reading radcliffe earnings first five potter films held trust fund able touch despite growing fame riches actor says keeping feet firmly ground people always looking say star goes rails told reporters 

In [120]:
true_k = 20

model = KMeans(
    
               n_clusters = true_k, 
               init = "k-means++", 
               max_iter = 100, 
            n_init = 1
)

model.fit(vectors)

order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()

In [121]:
with open ("results.txt", "w", encoding = "utf-8") as f:
    for i in range(true_k):
        f.write(f"Cluster {i}")
        f.write("\n")
        for ind in order_centroids[i, :10]:
            f.write('%s' % terms[ind],)
            f.write("\n")
        f.write("\n")
        f.write("\n")